In [72]:
import numpy as np

class Particle:
    def __init__(self, dim, minx, maxx):
        self.position = np.random.uniform(minx, maxx, dim)
        self.velocity = np.random.uniform(-1, 1, size=dim)
        self.pbest_position = self.position.copy()
        self.pbest_score = np.inf
        self.score = np.inf
        self.dim=dim
        
    def update_velocity(self, inertia, c1, c2, gbest_position,repulsive_sources, 
                        repulsion_strength=1.0,attraction_scale=1.0, epilson=1e-6):

        r1 = np.random.rand(self.dim)
        r2 = np.random.rand(self.dim)
        r3 = np.random.rand(self.dim)

        dp = self.pbest_position - self.position
        distance_p = np.linalg.norm(dp)
        # 高斯衰减系数
        cognitive_weight = c1 * np.exp(-distance_p** 2 / (2 * attraction_scale**2))
        cognitive = cognitive_weight * r1* dp 

        dg = gbest_position - self.position
        distance_g = np.linalg.norm(dg)
        social_weight = c2 * np.exp(-distance_g**2/ (2 * attraction_scale**2))
        social = social_weight * r2* dg 
        
        repulsion = np.zeros_like(self.velocity)
        for source in repulsive_sources:
            vec = self.position - source
            distance_v = np.linalg.norm(vec)
            if distance_v < epilson:
                continue
            repulsion_strength_scaled =  np.exp(-distance_v** 2 / (2 * attraction_scale**2))
            repulsion += (repulsion_strength_scaled) * r3* (vec)
            
        self.velocity = inertia * self.velocity + cognitive + social + repulsion_strength*repulsion

    def update_position(self, minx, maxx):
        self.position += self.velocity
        self.position = np.clip(self.position, minx, maxx)

    def mutation(self,pm):
        if np.random.rand()<pm:
            theta=1.0/(1.0+np.exp(-self.score))
            self.position+=np.random.normal(0,theta,size=self.position.shape)
    
def hybridize(parent1, parent2, dim, minx, maxx,pm=0.1):
    alpha = np.random.rand()
    child_pos = alpha * parent1.position + (1-alpha) * parent2.position
    child_vel = alpha * parent1.velocity + (1-alpha) * parent2.velocity
    if np.random.rand() < pm:
        child_pos+=np.random.normal(0,0.5,size=child_pos.shape)
    child = Particle(dim, minx, maxx)

    child.position = child_pos
    child.velocity = child_vel
    return child

def select_parents(remaining,k=2,replace=False):
    scores=np.array([p.score for p in remaining])
    fitness=1.0/(scores+1e-6)
    total_fitness=np.sum(fitness)
    probs=fitness/total_fitness
    parents_indices=np.random.choice(len(remaining),k,replace=replace,p=probs)
    return [remaining[i] for i in parents_indices]



def modified_pso(objective_func, dim, minx, maxx,
                 n_particles=50, n_iterations=200, inertia=0.8,pm=0.1, 
                 c1=1.5, c2=1.5, k_eliminate=2, attraction_scale=1.0):
    particles = [Particle(dim, minx, maxx) for _ in range(n_particles)]
    gbest_position = np.zeros(dim)
    gbest_score = np.inf

    for iteration in range(n_iterations):
        current_inertia = inertia - (inertia - 0.4) * (iteration / n_iterations)
        for p in particles:
            p.score = objective_func(p.position)
            if p.score < p.pbest_score:
                p.pbest_score = p.score
                p.pbest_position = p.position.copy()
            if p.score < gbest_score:
                gbest_score = p.score
                gbest_position = p.position.copy()
            
        particles.sort(key=lambda x: x.score, reverse=True)
        eliminated = particles[:k_eliminate]
        remaining = particles[k_eliminate:]

        new_particles = []
        for _ in range(k_eliminate):
            parents = select_parents(remaining, 2, replace=False)
            child = hybridize(parents[0], parents[1], dim, minx, maxx)
            new_particles.append(child)
        
        repulsive_sources = [e.position.copy() for e in eliminated]

        particles = remaining + new_particles
        for p in particles:
            p.update_velocity(current_inertia, c1, c2, gbest_position,
                            repulsive_sources,attraction_scale)
            p.update_position(minx, maxx)
            p.mutation(pm)


        print(f"Iteration {iteration}, Best Score: {gbest_score:.4f}")

    return gbest_position, gbest_score

def rastrigin(x):
    return 10*len(x) + np.sum(x**2 - 10*np.cos(2*np.pi*x))

if __name__ == "__main__":

    dim = 10
    n_particles = 80
    n_iterations = 200
    minx = -5.12
    maxx = 5.12
    inertia=0.8
    c1=c2=1.2
    k_eliminate=3
    pm=0.08

    best_position, best_score = modified_pso(
        rastrigin, dim, minx, maxx, n_particles, n_iterations,
        inertia,pm,c1,c2,k_eliminate,attraction_scale=2.0  # 可以调整这个参数来控制信息素衰减速度
    )
    print("Optimized Position:", best_position)
    print("Optimized Score:", best_score)

Iteration 0, Best Score: 106.1596
Iteration 1, Best Score: 106.1596
Iteration 2, Best Score: 95.1822
Iteration 3, Best Score: 95.1822
Iteration 4, Best Score: 90.4843
Iteration 5, Best Score: 90.4843
Iteration 6, Best Score: 90.4843
Iteration 7, Best Score: 90.4843
Iteration 8, Best Score: 90.4843
Iteration 9, Best Score: 90.4843
Iteration 10, Best Score: 90.4843
Iteration 11, Best Score: 90.3519
Iteration 12, Best Score: 85.3865
Iteration 13, Best Score: 83.6537
Iteration 14, Best Score: 83.5914
Iteration 15, Best Score: 83.5914
Iteration 16, Best Score: 83.0527
Iteration 17, Best Score: 82.4456
Iteration 18, Best Score: 82.1736
Iteration 19, Best Score: 82.0826
Iteration 20, Best Score: 82.0824
Iteration 21, Best Score: 82.0824
Iteration 22, Best Score: 82.0824
Iteration 23, Best Score: 82.0824
Iteration 24, Best Score: 73.3954
Iteration 25, Best Score: 73.3954
Iteration 26, Best Score: 73.0632
Iteration 27, Best Score: 73.0632
Iteration 28, Best Score: 73.0632
Iteration 29, Best Sco